In [1]:
import pandas as pd
import numpy as np

In [2]:
import re
import random
import copy

# Concatenate 2018 and 2019 Data

In [3]:
df_2018 = pd.read_csv("data/4_year/2018.csv")

In [4]:
df_2019 = pd.read_csv("data/4_year/2019.csv")

In [5]:
df_2018.isna().sum().sort_values(ascending=True).head(30)

UnitID                                                                0
parent/child_indicator_-_finance                                      0
historically_black_college_or_university                              0
institution_name                                                      0
institution_size_category                                             0
cohort                                                                0
city_location_of_institution                                          0
carnegie_classification_2018:_size_and_setting                        0
carnegie_classification_2018:_basic                                   0
sector_of_institution                                                 0
carnegie_classification_2018:_enrollment_profile                      0
graduation_rate_total_cohort                                          0
student-to-faculty_ratio                                              1
academic_support_expenses_as_a_percent_of_total_core_expenses   

In [6]:
df_2019.isna().sum().sort_values(ascending=True).head(30)

UnitID                                                                0
historically_black_college_or_university                              0
institution_name                                                      0
graduation_rate_total_cohort                                          0
institution_size_category                                             0
cohort                                                                0
city_location_of_institution                                          0
carnegie_classification_2018:_size_and_setting                        0
carnegie_classification_2018:_enrollment_profile                      0
carnegie_classification_2018:_basic                                   0
parent/child_indicator_-_finance                                      0
sector_of_institution                                                 0
research_expenses_as_a_percent_of_total_core_expenses                 4
academic_support_expenses_as_a_percent_of_total_core_expenses   

In [7]:
comparison = list(df_2018.columns == df_2019.columns)

In [8]:
differences = []
for i in range(len(comparison)):
    if comparison[i] == False:
        differences.append(i)
differences

[93, 95, 97, 104]

In [9]:
df_2019.columns[92]

'total_price_for_in-district_students_living_off_campus'

In [10]:
df_2018.columns[103]

'two_or_more_races_total_instructional_staff'

In [11]:
df_2019 = df_2019.rename(columns=lambda x: re.sub(r'_(\d){4}-(\d){2}', '', x))

In [12]:
df_2018 = df_2018.rename(columns=lambda x: re.sub(r'_(\d){4}-(\d){2}', '', x))
df_2018 = df_2018.rename(columns=lambda x: re.sub(r'__', '_', x))

In [13]:
print(df_2019.shape)
print(df_2018.shape)

(2315, 108)
(2286, 108)


In [14]:
total_df = pd.concat([df_2018, df_2019])

In [15]:
total_df.head()

,UnitID,academic_support_expenses_as_a_percent_of_total_core_expenses,act_composite_75th_percentile_score,act_english_75th_percentile_score,act_math_75th_percentile_score,act_writing_75th_percentile_score,american_indian_or_alaska_native_men_instructional_staff,american_indian_or_alaska_native_total_instructional_staff,american_indian_or_alaska_native_women_instructional_staff,asian_men_instructional_staff,...,total_women_age_25_and_over,total_women_age_25_and_under,transfer-out_rate_total_cohort,tuition_and_fees_as_a_percent_of_core_revenues,two_or_more_races_men_instructional_staff,two_or_more_races_total_instructional_staff,two_or_more_races_women_instructional_staff,white_men_instructional_staff,white_total_instructional_staff,white_women_instructional_staff
0,100654,5.0,19.0,19.0,18.0,NaN,0.0,1.0,1.0,17.0,...,NaN,NaN,42.0,30.0,0.0,0.0,0.0,17.0,32.0,15.0
1,100663,15.0,27.0,28.0,26.0,NaN,2.0,4.0,2.0,243.0,...,1636.0,4889.0,22.0,19.0,5.0,17.0,12.0,1011.0,1703.0,692.0
2,100690,7.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,60.0,90.0,0.0,0.0,0.0,26.0,30.0,4.0
3,100706,6.0,29.0,30.0,28.0,NaN,0.0,0.0,0.0,45.0,...,711.0,1974.0,30.0,30.0,0.0,1.0,1.0,125.0,245.0,120.0
4,100724,9.0,19.0,19.0,18.0,NaN,0.0,0.0,0.0,20.0,...,379.0,2675.0,37.0,34.0,0.0,0.0,0.0,30.0,54.0,24.0


# Save original dataframe to check the integrity of our filling methods

In [16]:
transposed = total_df.T

In [17]:
transposed.columns = transposed.loc["UnitID", :]

In [18]:
transposed.drop("UnitID", inplace=True)

In [19]:
check_integrity = list(transposed.isna().sum().sort_values().index[:5])

In [20]:
integrity_df = total_df.set_index("UnitID").loc[check_integrity, :].copy(deep=True)

In [21]:
integrity_df[["act_composite_75th_percentile_score", "graduation_rate_total_cohort"]]

,act_composite_75th_percentile_score,graduation_rate_total_cohort
UnitID,,
228769,25.0,49.0
228769,26.0,51.0
139755,32.0,87.0
139755,33.0,90.0
139959,30.0,86.0
139959,30.0,87.0
220862,25.0,48.0
220862,25.0,53.0
142115,25.0,46.0


# Filling Test Scores

In [22]:
# Horizontal Fill SAT Scores
sat_scores = total_df.filter(regex=("sat.*percentile")).columns.to_list()
act_scores = total_df.filter(regex=("act.*percentile")).columns.to_list()

In [23]:
# first preserve the fact that the scores are nan
for i in act_scores:
    name = i + "_isnan"
    total_df[name] = total_df[i].isna() * 1
    
for i in sat_scores:
    name = i + "_isnan"
    total_df[name] = total_df[i].isna() * 1

In [24]:
# define a variable called test_scores that holds column names of act and sat scores
test_scores = total_df.filter(regex=("((sat|act).*score(?!_isnan))")).columns

In [25]:
# sort my dataframe by
total_df.sort_values(["UnitID"], inplace=True)

In [26]:
id_group = total_df.groupby("UnitID")

In [27]:
# ffill and bfill is used to make sure if the observation above or below has a value it will fill
for i in test_scores:
    total_df[i] = id_group[i].transform(lambda group: group.ffill().bfill())

In [28]:
total_df.filter(regex=("act.*percentile|sat.*percentile")).isna().sum()

act_composite_75th_percentile_score                 1924
act_english_75th_percentile_score                   2242
act_math_75th_percentile_score                      2238
act_writing_75th_percentile_score                   3902
sat_critical_reading_75th_percentile_score          1984
sat_math_75th_percentile_score                      1974
sat_writing_75th_percentile_score                   2873
act_composite_75th_percentile_score_isnan              0
act_english_75th_percentile_score_isnan                0
act_math_75th_percentile_score_isnan                   0
act_writing_75th_percentile_score_isnan                0
sat_critical_reading_75th_percentile_score_isnan       0
sat_math_75th_percentile_score_isnan                   0
sat_writing_75th_percentile_score_isnan                0
dtype: int64

In [29]:
total_df[sat_scores[0]].isna().sum()

1984

In [30]:
for i in sat_scores:
    others = copy.deepcopy(sat_scores)
    others.remove(i)
    for j in others:
        total_df[i] = total_df[i].combine_first(total_df[j])
        total_df[j] = total_df[j].combine_first(total_df[i])

In [31]:
for i in act_scores:
    others = copy.deepcopy(act_scores)
    others.remove(i)
    for j in others:
        total_df[i] = total_df[i].combine_first(total_df[j])
        total_df[j] = total_df[j].combine_first(total_df[i])

In [32]:
total_df.filter(regex=("act.*percentile|sat.*percentile")).isna().sum()

act_composite_75th_percentile_score                 1916
act_english_75th_percentile_score                   1916
act_math_75th_percentile_score                      1916
act_writing_75th_percentile_score                   1916
sat_critical_reading_75th_percentile_score          1972
sat_math_75th_percentile_score                      1972
sat_writing_75th_percentile_score                   1972
act_composite_75th_percentile_score_isnan              0
act_english_75th_percentile_score_isnan                0
act_math_75th_percentile_score_isnan                   0
act_writing_75th_percentile_score_isnan                0
sat_critical_reading_75th_percentile_score_isnan       0
sat_math_75th_percentile_score_isnan                   0
sat_writing_75th_percentile_score_isnan                0
dtype: int64

## ACT to SAT Conversions

SAT to ACT conversion table taken from [The Princeton Review]("https://www.princetonreview.com/college-advice/act-to-sat-conversion")

In [33]:
# Create a conversion from act scores to SAT scores

In [34]:
ACT_SAT = pd.read_html("https://www.princetonreview.com/college-advice/act-to-sat-conversion")

In [35]:
act_sat = ACT_SAT[0]

In [36]:
act_sat.head()

,SAT,ACT,Unnamed: 2,SAT.1,ACT.1,Unnamed: 5,SAT.2,ACT.2
0,1600,36,NaN,1250,26,NaN,900,17
1,1590,35,NaN,1240,26,NaN,890,16
2,1580,35,NaN,1230,25,NaN,880,16
3,1570,35,NaN,1220,25,NaN,870,16
4,1560,35,NaN,1210,25,NaN,860,16


In [37]:
act_sat = act_sat.drop(["Unnamed: 2", "Unnamed: 5"], axis=1)

In [38]:
act_sat.head(30)

,SAT,ACT,SAT.1,ACT.1,SAT.2,ACT.2
0,1600,36,1250,26,900,17
1,1590,35,1240,26,890,16
2,1580,35,1230,25,880,16
3,1570,35,1220,25,870,16
4,1560,35,1210,25,860,16
5,1550,34,1200,25,850,15
6,1540,34,1190,24,840,15
7,1530,34,1180,24,830,15
8,1520,34,1170,24,820,15
9,1510,33,1160,24,810,15


In [39]:
def convert_to_act(x):
    if np.isnan(x):
        return np.nan
    
    x = int(x)
    if x < 630:
        return 11
    elif x < 720 / 20:
        return 12
    elif x < 760 / 2:
        return 13
    elif x < 810 / 2:
        return 14
    elif x < 860 / 2:
        return 15
    elif x < 900 / 2:
        return 16
    elif x < 940 / 2:
        return 17
    elif x < 960 / 2:
        return 18
    elif x < 1020 / 2:
        return 19
    elif x < 1060 / 2:
        return 20
    elif x < 1100 / 2:
        return 21
    elif x < 1130 / 2:
        return 22
    elif x < 1160 / 2:
        return 23
    elif x < 1200 / 2:
        return 24
    elif x < 1240 / 2:
        return 25
    elif x < 1280 / 2:
        return 26
    elif x < 1310 / 2:
        return 27
    elif x < 1350 / 2:
        return 28
    elif x < 1390 / 2:
        return 29
    elif x < 1420 / 2:
        return 30
    elif x < 1450 / 2:
        return 31
    elif x < 1490 / 2:
        return 32
    elif x < 1520 / 2:
        return 33
    elif x < 1560 / 2:
        return 34
    elif x < 1590 / 2:
        return 35
    elif x < 1600 / 2:
        return 36
        

In [40]:
def convert_to_sat(x):
    if np.isnan(x):
        return np.nan
    
    x = int(x)
    if x == 11:
        return 620 / 2
    elif x == 12:
        return 700 / 2
    elif x == 13:
        return 740 / 2
    elif x == 14:
        return 800 / 2
    elif x == 15:
        return 840 / 2
    elif x == 16:
        return 880 / 2
    elif x == 17:
        return 930 / 2
    elif x == 18:
        return 960 / 2
    elif x == 19:
        return 1000 / 2
    elif x == 20:
        return 1040 / 2
    elif x == 21:
        return 1090 / 2
    elif x == 22:
        return 1120 / 2
    elif x == 23:
        return 1150 / 2
    elif x == 24:
        return 1190 / 2
    elif x == 25:
        return 1230 / 2
    elif x == 26:
        return 1270 / 2
    elif x == 27:
        return 1280 / 2
    elif x == 28:
        return 1340 / 2
    elif x == 29:
        return 1380 / 2
    elif x == 30:
        return 1410 / 2
    elif x == 31:
        return 1440 / 2
    elif x == 32:
        return 1480 / 2
    elif x == 33:
        return 1510 / 2
    elif x == 34:
        return 1550 / 2
    elif x == 35:
        return 1580 / 2
    elif x == 36:
        return 1600 / 2
    

In [41]:
for i in act_scores:
    total_df[i] = total_df[i].combine_first(total_df[sat_scores[0]].apply(lambda x: convert_to_act(x))) 

for i in sat_scores:
    total_df[i] = total_df[i].combine_first(total_df[act_scores[0]].apply(lambda x: convert_to_sat(x)))

In [42]:
sat_scores, act_scores

(['sat_critical_reading_75th_percentile_score',
  'sat_math_75th_percentile_score',
  'sat_writing_75th_percentile_score'],
 ['act_composite_75th_percentile_score',
  'act_english_75th_percentile_score',
  'act_math_75th_percentile_score',
  'act_writing_75th_percentile_score'])

In [43]:
total_df.filter(regex=("act.*percentile|sat.*percentile")).isna().sum()

act_composite_75th_percentile_score                 1810
act_english_75th_percentile_score                   1810
act_math_75th_percentile_score                      1810
act_writing_75th_percentile_score                   1810
sat_critical_reading_75th_percentile_score          1810
sat_math_75th_percentile_score                      1810
sat_writing_75th_percentile_score                   1810
act_composite_75th_percentile_score_isnan              0
act_english_75th_percentile_score_isnan                0
act_math_75th_percentile_score_isnan                   0
act_writing_75th_percentile_score_isnan                0
sat_critical_reading_75th_percentile_score_isnan       0
sat_math_75th_percentile_score_isnan                   0
sat_writing_75th_percentile_score_isnan                0
dtype: int64

# Back fill and Sideways fill other Highly Null Columns

In [44]:
total_df.isna().sum().sort_values(ascending=False).head(10)

local_appropriations_as_a_percent_of_core_revenues           3169
state_appropriations_as_percent_of_core_revenues             3169
graduation_rate_native_hawaiian_or_other_pacific_islander    2755
total_women_age_25_and_over                                  2021
total_men_age_25_and_over                                    2021
grand_total_age_25_and_over                                  2021
total_men_age_25_and_under                                   2016
grand_total_age_25_and_under                                 2016
total_women_age_25_and_under                                 2016
transfer-out_rate_total_cohort                               1909
dtype: int64

In [45]:
x_cols = total_df.filter(regex="^(?!graduation_rate).").columns

In [46]:
cols_to_fill = total_df.loc[:, x_cols].columns

In [47]:
id_group[cols_to_fill[3]].transform(lambda group: group.ffill().bfill()).index

Int64Index([   0,    0,    1,    1,    2,    2,    3,    3,    4,    4,
            ...
            2308, 2309, 2283, 2310, 2284, 2311, 2285, 2312, 2313, 2314],
           dtype='int64', length=4601)

In [48]:
for i in cols_to_fill:
    total_df[i] = id_group[i].transform(lambda group: group.ffill().bfill())

In [49]:
total_df.isna().sum().sort_values(ascending=True).head(40)

UnitID                                                                                                    0
sat_critical_reading_75th_percentile_score_isnan                                                          0
act_writing_75th_percentile_score_isnan                                                                   0
act_math_75th_percentile_score_isnan                                                                      0
act_english_75th_percentile_score_isnan                                                                   0
act_composite_75th_percentile_score_isnan                                                                 0
sector_of_institution                                                                                     0
parent/child_indicator_-_finance                                                                          0
sat_math_75th_percentile_score_isnan                                                                      0
institution_size_category   

### This is just to check the integrity of our data frame

In [50]:
total_df.set_index("UnitID").loc[check_integrity, ["total_price_for_out-of-state_students_living_on_campus","graduation_rate_total_cohort"]]

,total_price_for_out-of-state_students_living_on_campus,graduation_rate_total_cohort
UnitID,,
228769,31864.0,51.0
228769,30408.0,49.0
139755,41558.0,87.0
139755,44052.0,90.0
139959,39460.0,86.0
139959,40890.0,87.0
220862,35902.0,53.0
220862,37525.0,48.0
142115,32612.0,51.0


In [51]:
integrity_df[["total_price_for_out-of-state_students_living_on_campus","graduation_rate_total_cohort"]]

,total_price_for_out-of-state_students_living_on_campus,graduation_rate_total_cohort
UnitID,,
228769,30408.0,49.0
228769,31864.0,51.0
139755,41558.0,87.0
139755,44052.0,90.0
139959,39460.0,86.0
139959,40890.0,87.0
220862,37525.0,48.0
220862,35902.0,53.0
142115,31658.0,46.0


In [52]:
total_df.state_abbreviation.fillna("ismissing", inplace=True)

### Before saving to disk we want to preserve the fact that some observations are null

In [53]:
# this shows that they are all na in the same areas
print(total_df.filter(regex="percent_of_total_enrollment").isna().sum())
print("\nALL :")
print((total_df.filter(regex="percent_of_total_enrollment").isna().sum(axis=1) == 11).sum())

percent_of_total_enrollment_that_are_american_indian_or_alaska_native             84
percent_of_total_enrollment_that_are_asian                                        84
percent_of_total_enrollment_that_are_asian/native_hawaiian/pacific_islander       84
percent_of_total_enrollment_that_are_black_or_african_american                    84
percent_of_total_enrollment_that_are_hispanic/latino                              84
percent_of_total_enrollment_that_are_native_hawaiian_or_other_pacific_islander    84
percent_of_total_enrollment_that_are_nonresident_alien                            84
percent_of_total_enrollment_that_are_race/ethnicity_unknown                       84
percent_of_total_enrollment_that_are_two_or_more_races                            84
percent_of_total_enrollment_that_are_white                                        84
percent_of_total_enrollment_that_are_women                                        84
dtype: int64

ALL :
84


In [54]:
total_df.filter(regex="act").isna().sum()

act_composite_75th_percentile_score                                 1810
act_english_75th_percentile_score                                   1810
act_math_75th_percentile_score                                      1810
act_writing_75th_percentile_score                                   1810
government_grants_and_contracts_as_a_percent_of_core_revenues          7
private_gifts_grants_and_contracts_as_a_percent_of_core_revenues       7
act_composite_75th_percentile_score_isnan                              0
act_english_75th_percentile_score_isnan                                0
act_math_75th_percentile_score_isnan                                   0
act_writing_75th_percentile_score_isnan                                0
dtype: int64

In [55]:
#Lets do an isna for all of the main categories of data where there are a significant amount of null values
total_df["percent_of_total_enrollment_isnan"] = total_df["percent_of_total_enrollment_that_are_asian"].isna() * 1
total_df["grand_total_age_25_and_over_isnan"] = total_df["grand_total_age_25_and_over"].isna() * 1
total_df["grand_total_instructional_staff_isnan"] = total_df["grand_total_instructional_staff"].isna() * 1
total_df["total_price_for_in-district_students_living_on_campus"] = total_df["total_price_for_in-district_students_living_on_campus"]

In [56]:
total_df.filter(regex="isnan")

,act_composite_75th_percentile_score_isnan,act_english_75th_percentile_score_isnan,act_math_75th_percentile_score_isnan,act_writing_75th_percentile_score_isnan,sat_critical_reading_75th_percentile_score_isnan,sat_math_75th_percentile_score_isnan,sat_writing_75th_percentile_score_isnan,percent_of_total_enrollment_isnan,grand_total_age_25_and_over_isnan,grand_total_instructional_staff_isnan
0,0,0,0,1,0,0,1,0,1,0
0,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0
1,0,0,0,1,0,0,1,0,0,0
2,1,1,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2311,1,1,1,1,1,1,1,1,1,1
2285,1,1,1,1,1,1,1,1,1,1
2312,1,1,1,1,1,1,1,1,1,1
2313,1,1,1,1,1,1,1,1,1,1


In [57]:
total_df.to_csv("data/4_year/aggregate.csv")

# Imputation Pipeline

In [59]:
from sklearn.impute import KNNImputer

In [60]:
imputer = KNNImputer()

In [61]:
total_df.set_index("UnitID", inplace=True)

In [62]:
numeric = total_df.select_dtypes(['float', 'int'])

In [63]:
num_cols = numeric.columns

In [64]:
total_df.loc[:, num_cols] = imputer.fit_transform(numeric)

In [65]:
total_df.isna().sum()

academic_support_expenses_as_a_percent_of_total_core_expenses    0
act_composite_75th_percentile_score                              0
act_english_75th_percentile_score                                0
act_math_75th_percentile_score                                   0
act_writing_75th_percentile_score                                0
                                                                ..
sat_math_75th_percentile_score_isnan                             0
sat_writing_75th_percentile_score_isnan                          0
percent_of_total_enrollment_isnan                                0
grand_total_age_25_and_over_isnan                                0
grand_total_instructional_staff_isnan                            0
Length: 117, dtype: int64

In [66]:
total_df.to_csv("data/4_year/eda.csv")